In [3]:
import pandas as pd
%pip install langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone docx2txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import dotenv

dotenv.load_dotenv()

True

In [5]:
%pip install langchain_upstage

Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
from langchain_upstage import UpstageEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from pprint import pprint

# os.environ["GPT_API_KEY"] = GPT_API_KEY
UPSTAGE_API_KEY = os.environ["UPSTAGE_API_KEY"]
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

In [7]:
folder_path = '../data'

In [8]:
document_list = []

for file in os.listdir(folder_path):
    print(file)
    temp_loader = CSVLoader(file_path=f"{folder_path}/{file}", encoding='utf-8-sig')
    temp_document_list = temp_loader.load_and_split(text_splitter=text_splitter)
    
    document_list.extend(temp_document_list)

print(len(document_list))

CANCEL_main_qna_list.csv
DELIVERY_main_qna_list.csv
ORDER_main_qna_list.csv
REFUND_main_qna_list.csv
SUB_qna_list.csv
309


In [9]:
# Upstage 에서 제공하는 Embedding Model을 활용
embedding = UpstageEmbeddings(model="solar-embedding-1-large",
                              api_key=UPSTAGE_API_KEY)

In [10]:
from langchain_pinecone import PineconeVectorStore


index_name = 'upstage-index'

c:\Users\Admin\anaconda3\envs\project_final\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [13]:
# DB 처음 만들 때
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)


In [11]:
# 만들어 놓은 DB가 있을 때
database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

# Vectorstore 유사도 검색

In [22]:
query = '반품비는 얼마인가요?'

results = database.similarity_search_with_score(query=query, k=3)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")
    

* [SIM=0.488640] Question: 단순 변심 반품 비용
Answer: 로켓배송 상품
• 와우 멤버십 회원 : 무료
• 와우 멤버십 비회원
ㆍ총 주문금액 - 반품 상품금액 = 19800원 미만인 경우 반품비 5000원 차감 후 환불
ㆍ총 주문금액 - 반품 상품금액 = 19800원 이상인 경우 반품비 2500원 차감 후 환불
 로켓직구 상품
• 와우 멤버십 회원 : 5000원 (미국 직구 2600원)
• 와우 멤버십 비회원
ㆍ총 주문금액 - 반품 상품금액 = 29800원 미만인 경우 반품비 5000원 차감 후 환불 (미국 직구 2600원)
ㆍ총 주문금액 - 반품 상품금액 = 29800원 이상인 경우 반품비 2500원 차감 후 환불 (미국 직구 2500원)
• 단 TV 및 일부 전자기기와 같은 특정 제품(다이슨 로봇청소기 등)은 2600원/5000원 보다 높은 교환/반품비가 적용될 수 있음
 판매자 배송 상품
• 배송 시작 이후 최초 배송비를 포함한 왕복 배송비 차감 후 환불
• 반품비는 판매자 및 상품에 따라 상이함
• 도서산간 지역 설치 상품 업체에서 직접 배송하는 상품의 경우 반품비가 추가될 수 있음
keywords: SUB
count: 0 [{'row': 15.0, 'source': '../data/SUB_qna_list.csv'}]
* [SIM=0.488591] Question: 단순 변심 반품 비용
Answer: 로켓배송 상품
• 와우 멤버십 회원 : 무료
• 와우 멤버십 비회원
ㆍ총 주문금액 - 반품 상품금액 = 19800원 미만인 경우 반품비 5000원 차감 후 환불
ㆍ총 주문금액 - 반품 상품금액 = 19800원 이상인 경우 반품비 2500원 차감 후 환불
 로켓직구 상품
• 와우 멤버십 회원 : 5000원 (미국 직구 2600원)
• 와우 멤버십 비회원
ㆍ총 주문금액 - 반품 상품금액 = 29800원 미만인 경우 반품비 5000원 차감 후 환불 (미국 직구 2600원)
ㆍ총 주문금액 - 반품 상품금액 = 29800원 이상인 경우 반품

# LLM 질의 테스트

In [46]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retriever = database.as_retriever(
    search_type="mmr", search_kwargs={"k": 3, "fetch_k": 5}
)

template ='''
[context]: {context}
---
[질의]: {query}

위의 [context] 정보 내에서 [질의]에 대해 상담사 입장에서 사용자가 만족할 수 있을 정도로 성의있고 친절하게 답하세요.
단, [context] 정보에 없는 내용을 답해서는 안됩니다. 쉼표로 끊어서 대답하기 보다는, 최대한 온점으로 문장을 끊어주세요.
이 모든 정보를 종합해서 2~3줄의 구어체로 답해주세요. 이유는 필요없어. 상냥하고 전문적으로 응대만 해.

[context]를 받고, 관련된 내용을 토대로 상담합니다. 가능한 한 직접 정보를 확인하고 해결합니다.
고객에게 직접 확인하게 하지 말고, 직접 확인 후 안내하는 방식으로 진행합니다.

양해 멘트 및 쿠션어를 많이 사용합니다.
파손 등 부정적인 내용의 문의사항일 경우, "불편을 드려 죄송합니다" 같은 쿠션어를 사용해 주세요.
고객이 직접 작업을 수행해야 하는 경우, 문장 앞에 "번거로우시겠지만" 같은 쿠션어를 사용해 주세요.
"안 된다", "불가능하다" 대신 "어렵습니다"처럼 간접적인 부정어를 사용합니다.

추가 정보가 필요할 때에는 정보 확인 멘트를 사용해서 응대합니다.
조건별로 안내 내용이 다른 경우,양해 멘트와 정보 확인 멘트의 구성으로 답 합니다.
정보 확인 멘트는 "정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다."라는 멘트를 사용합니다.

예시
[질의] 반품비는 얼마인가요?
[답] (1차 안내) 귀책 사유에 따라 반품비 발생 여부나 비용에 차이가 있을 수 있습니다. (정보 확인) 정확한 상담을 위해 주문하신 주문 번호 확인 부탁드립니다.

'''
prompt = ChatPromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

def merge_pages(pages):
    merged = "\n\n".join(page.page_content for page in pages)
    return merged

chain = (
    {"query": RunnablePassthrough(), "context": retriever | merge_pages}
    | prompt
    | llm
    | StrOutputParser()
)

query = '배송 중 상품을 다른 주소로 변경할 수 있나요?'

results = database.similarity_search_with_score(query=query, k=3)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

pprint(results)

answer = chain.invoke(query).replace('  ', ' ').split('.')
print("Answer : ", end='')
for ans in answer:
    print(ans + '.')
    

* [SIM=0.538316] : 38
Question: Q[배송지] 배송 중에 배송지 및 배송요청사항을 변경할 수 있나요?
Answer: 아니요. 상품이 이미 출고되어 배송 중인 상태에서는 배송지 및 배송요청사항을 변경할 수 없습니다.
keywords: DELIVERY
count: 0 [{'row': 38.0, 'source': '../data/DELIVERY_main_qna_list.csv'}]
* [SIM=0.538260] : 38
Question: Q[배송지] 배송 중에 배송지 및 배송요청사항을 변경할 수 있나요?
Answer: 아니요. 상품이 이미 출고되어 배송 중인 상태에서는 배송지 및 배송요청사항을 변경할 수 없습니다.
keywords: DELIVERY
count: 0 [{'row': 38.0, 'source': '../data/DELIVERY_main_qna_list.csv'}]
* [SIM=0.510390] : 39
Question: Q[배송지] 교환 신청 시 새 상품을 받을 배송지를 변경할 수 있나요?
Answer: 네 마이쿠팡에서 교환 신청 시 회수지와 배송지 주소를 다르게 정할 수 있습니다.
keywords: CANCEL
count: 0 [{'row': 39.0, 'source': '../data/CANCEL_main_qna_list.csv'}]
[(Document(id='18bed970-4bb3-4465-b37a-60f11e602eef', metadata={'row': 38.0, 'source': '../data/DELIVERY_main_qna_list.csv'}, page_content=': 38\nQuestion: Q[배송지] 배송 중에 배송지 및 배송요청사항을 변경할 수 있나요?\nAnswer: 아니요. 상품이 이미 출고되어 배송 중인 상태에서는 배송지 및 배송요청사항을 변경할 수 없습니다.\nkeywords: DELIVERY\ncount: 0'),
  0.538315892),
 (Document(id='8a6afc89-6b94-4